<a href="https://colab.research.google.com/github/TyroneNorth/Data-Science/blob/master/Big_Mart_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

#Exploration

In [2]:
#Load dataset
train = pd.read_csv('test_AbJTz2l.csv')
test = pd.read_csv('train_v9rqX0R.csv')

#combining datasets for feature engineering
train['source'] = 'train'
test['source'] = 'test'
data = pd.concat([train, test], ignore_index = True)
print(train.shape, '', test.shape, '', data.shape)

(5681, 12)  (8523, 13)  (14204, 13)


In [3]:
#Number of missing values per column
data.isnull().sum()

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
source                          0
Item_Outlet_Sales            5681
dtype: int64

In [4]:
data.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,11765.000000,14204.000000,14204.000000,14204.000000,8523.000000
mean,12.792854,0.065953,141.004977,1997.830681,2181.288914
std,4.652502,0.051459,62.086938,8.371664,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.710000,0.027036,94.012000,1987.000000,834.247400
50%,12.600000,0.054021,142.247000,1999.000000,1794.331000
75%,16.750000,0.094037,185.855600,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [5]:
#Checking number of unique values per column
data.apply(lambda x: len(x.unique()))

Item_Identifier               1559
Item_Weight                    416
Item_Fat_Content                 5
Item_Visibility              13006
Item_Type                       16
Item_MRP                      8052
Outlet_Identifier               10
Outlet_Establishment_Year        9
Outlet_Size                      4
Outlet_Location_Type             3
Outlet_Type                      4
source                           2
Item_Outlet_Sales             3494
dtype: int64

In [6]:
#Filter categorical variables
categorical_columns = [x for x in data.dtypes.index if data.dtypes[x]=='object']
#Exclude ID cols and source:
categorical_columns = [x for x in categorical_columns if x not in ['Item_Identifier','Outlet_Identifier','source']]
#Print frequency of categories
for col in categorical_columns:
  print('\nFrequency of Categories for varible %s'%col)
  print(data[col].value_counts())


Frequency of Categories for varible Item_Fat_Content
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Frequency of Categories for varible Item_Type
Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

Frequency of Categories for varible Outlet_Size
Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: int64

Frequency of Categories for varible Outlet_Location_Type
Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64

F

### Some things to Note: Lowfat and LF are same; 
### Categories like hard drink and soft drink can be combined

In [7]:
#Combining Categories
item_type = data['Item_Type'].values.tolist()
i = 0
for i in range(len(item_type)):
  if item_type[i] == 'Hard Drinks' or item_type[i] == 'Soft Drinks':
    item_type[i] = 'Drinks'
  elif item_type[i] == 'Canned' or item_type[i] == 'Starchy Foods' or item_type[i] == 'Frozen Foods' or item_type[i] == 'Fruits and Vegetables' or item_type[i] == 'Meat' or item_type[i] == 'Seafood' or item_type[i] == 'Breakfast' or item_type[i] == 'Snack Foods' or item_type[i] == 'Dairy' or item_type[i] == 'Breads':
    item_type[i] = 'Food'
  elif item_type[i] == 'Others' or item_type[i] == 'Health and Hygiene' or item_type[i] == 'Baking Goods' or item_type[i] == 'Household':
    item_type[i] = 'Non Persihable'

data['Item_Type'] = item_type

#Data Cleaning

In [8]:
#Imputing missing values
imp = SimpleImputer(missing_values = np.nan, strategy = 'mean')
#transforming np.nan values to mean value of Item Outlet Sales and Outlet Weight
item_avg_sales = imp.fit_transform(data[['Item_Outlet_Sales']])
item_weight = imp.fit_transform(data[['Item_Weight']])
#output new table
item_avg_sales

array([[2181.28891358],
       [2181.28891358],
       [2181.28891358],
       ...,
       [1193.1136    ],
       [1845.5976    ],
       [ 765.67      ]])

In [9]:
item_weight

array([[20.75],
       [ 8.3 ],
       [14.6 ],
       ...,
       [10.6 ],
       [ 7.21],
       [14.8 ]])

In [10]:
#Updating Columns
data['Item_Outlet_Sales'] = item_avg_sales
data['Item_Weight'] = item_weight

In [11]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,source,Item_Outlet_Sales
0,FDW58,20.750000,Low Fat,0.007565,Food,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1,train,2181.288914
1,FDW14,8.300000,reg,0.038428,Food,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1,train,2181.288914
2,NCN55,14.600000,Low Fat,0.099575,Non Persihable,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store,train,2181.288914
3,FDQ58,7.315000,Low Fat,0.015388,Food,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1,train,2181.288914
4,FDY38,12.792854,Regular,0.118599,Food,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3,train,2181.288914


In [12]:
data.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
source                          0
Item_Outlet_Sales               0
dtype: int64

In [13]:
#Outlet Size column still has missing values
imp = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
outlet_sizes = imp.fit_transform(data[['Outlet_Size']])
outlet_sizes

array([['Medium'],
       ['Medium'],
       ['Medium'],
       ...,
       ['Small'],
       ['Medium'],
       ['Small']], dtype=object)

In [14]:
#Updating Column
data['Outlet_Size'] = outlet_sizes

In [15]:
data['Item_Fat_Content'].unique()
#Low Fat, LF, and low fat are all the same
#As well as 'reg and 'Regular'

array(['Low Fat', 'reg', 'Regular', 'LF', 'low fat'], dtype=object)

In [16]:
#converting LF and low fat to 'Low Fat'
fat = data['Item_Fat_Content'].values.tolist()

i = 0
for i in range(len(fat) - 1):
  if fat[i] == 'low fat' or fat[i] == 'LF':
    fat[i] = 'Low Fat'
  elif fat[i] == 'reg':
    fat[i] = 'Regular'

fat

#Alternative method
#data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF' : 'Low Fat',
#                                                             'reg' : 'Regular',
#                                                             'low fat' : 'Low Fat'})

['Low Fat',
 'Regular',
 'Low Fat',
 'Low Fat',
 'Regular',
 'Regular',
 'Regular',
 'Low Fat',
 'Regular',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Regular',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Regular',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Regular',
 'Low Fat',
 'Regular',
 'Low Fat',
 'Regular',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Regular',
 'Regular',
 'Regular',
 'Low Fat',
 'Regular',
 'Low Fat',
 'Regular',
 'Regular',
 'Regular',
 'Regular',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Regular',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Regular',
 'Low Fat',
 'Low Fat',
 'Regular',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Low Fat',
 'Regular',
 'Low Fat',
 'Re

In [17]:
#Parsing the transformed fat content column
data['Item_Fat_Content'] = fat

In [18]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,source,Item_Outlet_Sales
0,FDW58,20.750000,Low Fat,0.007565,Food,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1,train,2181.288914
1,FDW14,8.300000,Regular,0.038428,Food,87.3198,OUT017,2007,Medium,Tier 2,Supermarket Type1,train,2181.288914
2,NCN55,14.600000,Low Fat,0.099575,Non Persihable,241.7538,OUT010,1998,Medium,Tier 3,Grocery Store,train,2181.288914
3,FDQ58,7.315000,Low Fat,0.015388,Food,155.0340,OUT017,2007,Medium,Tier 2,Supermarket Type1,train,2181.288914
4,FDY38,12.792854,Regular,0.118599,Food,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3,train,2181.288914


In [19]:
data.isnull().sum()
#No missing data

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
source                       0
Item_Outlet_Sales            0
dtype: int64

#The next step is to transform the data to from str so that the estimator can work with the data

In [20]:
data.shape

(14204, 13)

In [21]:
data.info()
#Each object type needs to be transformed

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            14204 non-null  object 
 1   Item_Weight                14204 non-null  float64
 2   Item_Fat_Content           14204 non-null  object 
 3   Item_Visibility            14204 non-null  float64
 4   Item_Type                  14204 non-null  object 
 5   Item_MRP                   14204 non-null  float64
 6   Outlet_Identifier          14204 non-null  object 
 7   Outlet_Establishment_Year  14204 non-null  int64  
 8   Outlet_Size                14204 non-null  object 
 9   Outlet_Location_Type       14204 non-null  object 
 10  Outlet_Type                14204 non-null  object 
 11  source                     14204 non-null  object 
 12  Item_Outlet_Sales          14204 non-null  float64
dtypes: float64(4), int64(1), object(8)
memory usag

In [22]:
print(' ',data.Item_Fat_Content.unique(),
      '\n ', data.Item_Type.unique(),
      ' \n ', data.Outlet_Type.unique(),
      ' \n ', data.Outlet_Location_Type.unique(),
      ' \n ', data.Outlet_Identifier.unique())

  ['Low Fat' 'Regular'] 
  ['Food' 'Non Persihable' 'Drinks']  
  ['Supermarket Type1' 'Grocery Store' 'Supermarket Type3'
 'Supermarket Type2']  
  ['Tier 1' 'Tier 2' 'Tier 3']  
  ['OUT049' 'OUT017' 'OUT010' 'OUT027' 'OUT046' 'OUT018' 'OUT045' 'OUT019'
 'OUT013' 'OUT035']


In [23]:
X = data.drop(['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year', 'Item_Outlet_Sales', 'source', 'Outlet_Size', 'Item_Identifier']
              , axis = 'columns')

#Column Transformer is useful for different transformation on different columns
ct = make_column_transformer((OneHotEncoder(sparse = False), ['Item_Fat_Content', 'Item_Type', 'Outlet_Type', 'Outlet_Location_Type', 'Outlet_Identifier']), remainder = 'passthrough')

transformed = ct.fit_transform(X)
transformed

array([[1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 1., 0.]])

In [24]:
#Encoding ordinal values
enc = OrdinalEncoder()
outlet_transformed = enc.fit_transform(data[['Outlet_Size']])
outlet_transformed

array([[1.],
       [1.],
       [1.],
       ...,
       [2.],
       [1.],
       [2.]])

In [25]:
outlet_df = pd.DataFrame(data = outlet_transformed, columns = ['Outlet_Size'])
transformed_df = pd.DataFrame(data = transformed, columns = ['Low Fat', 'Regular', 'Foods', 'Drinks', 'Non Perishable', 'Supermarket Type1', 'Grocery Store', 'Supermarket Type3',
 'Supermarket Type2', 'Tier 1', 'Tier 2', 'Tier 3', 'OUT049', 'OUT017', 'OUT010', 'OUT027', 'OUT046', 'OUT018', 'OUT045', 'OUT019', 'OUT013', 'OUT035'])

In [26]:
outlet_df

,Outlet_Size
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
14199,0.0
14200,1.0
14201,2.0
14202,1.0


In [27]:
transformed_df

,Low Fat,Regular,Foods,Drinks,Non Perishable,Supermarket Type1,Grocery Store,Supermarket Type3,Supermarket Type2,Tier 1,Tier 2,Tier 3,OUT049,OUT017,OUT010,OUT027,OUT046,OUT018,OUT045,OUT019,OUT013,OUT035
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14199,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14200,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
14201,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
14202,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
transformed_df.columns

Index(['Low Fat', 'Regular', 'Foods', 'Drinks', 'Non Perishable',
       'Supermarket Type1', 'Grocery Store', 'Supermarket Type3',
       'Supermarket Type2', 'Tier 1', 'Tier 2', 'Tier 3', 'OUT049', 'OUT017',
       'OUT010', 'OUT027', 'OUT046', 'OUT018', 'OUT045', 'OUT019', 'OUT013',
       'OUT035'],
      dtype='object')

In [29]:
data.drop(labels = ['Item_Fat_Content', 'Item_Type', 'Outlet_Type', 'Outlet_Location_Type', 'Outlet_Identifier'], inplace = True, axis = 'columns')

In [30]:
data.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year', 'Outlet_Size', 'source',
       'Item_Outlet_Sales'],
      dtype='object')

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            14204 non-null  object 
 1   Item_Weight                14204 non-null  float64
 2   Item_Visibility            14204 non-null  float64
 3   Item_MRP                   14204 non-null  float64
 4   Outlet_Establishment_Year  14204 non-null  int64  
 5   Outlet_Size                14204 non-null  object 
 6   source                     14204 non-null  object 
 7   Item_Outlet_Sales          14204 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 887.9+ KB


In [32]:
transformed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Low Fat            14204 non-null  float64
 1   Regular            14204 non-null  float64
 2   Foods              14204 non-null  float64
 3   Drinks             14204 non-null  float64
 4   Non Perishable     14204 non-null  float64
 5   Supermarket Type1  14204 non-null  float64
 6   Grocery Store      14204 non-null  float64
 7   Supermarket Type3  14204 non-null  float64
 8   Supermarket Type2  14204 non-null  float64
 9   Tier 1             14204 non-null  float64
 10  Tier 2             14204 non-null  float64
 11  Tier 3             14204 non-null  float64
 12  OUT049             14204 non-null  float64
 13  OUT017             14204 non-null  float64
 14  OUT010             14204 non-null  float64
 15  OUT027             14204 non-null  float64
 16  OUT046             142

In [33]:
def df_append(column_list = list, df1 = pd.DataFrame, df2 = pd.DataFrame):
  """Appends columns of one DataFrame to another.
   Takes a list of column names to append and the two datasets to append from and to"""
  for col in column_list:
    df1[col] = df2[col]

df_append(data.columns.to_list(), transformed_df, data)

In [34]:
transformed_df['Outlet_Size'] = outlet_transformed

In [35]:
transformed_df

,Low Fat,Regular,Foods,Drinks,Non Perishable,Supermarket Type1,Grocery Store,Supermarket Type3,Supermarket Type2,Tier 1,Tier 2,Tier 3,OUT049,OUT017,OUT010,OUT027,OUT046,OUT018,OUT045,OUT019,OUT013,OUT035,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,source,Item_Outlet_Sales
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,FDW58,20.750000,0.007565,107.8622,1999,1.0,train,2181.288914
1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FDW14,8.300000,0.038428,87.3198,2007,1.0,train,2181.288914
2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NCN55,14.600000,0.099575,241.7538,1998,1.0,train,2181.288914
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FDQ58,7.315000,0.015388,155.0340,2007,1.0,train,2181.288914
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,FDY38,12.792854,0.118599,234.2300,1985,1.0,train,2181.288914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14199,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FDF22,6.865000,0.056783,214.5218,1987,0.0,test,2778.383400
14200,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,FDS36,8.380000,0.046982,108.1570,2002,1.0,test,549.285000
14201,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NCJ29,10.600000,0.035186,85.1224,2004,2.0,test,1193.113600
14202,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,FDN46,7.210000,0.145221,103.1332,2009,1.0,test,1845.597600


In [36]:
#Creating a column for number of operating years
transformed_df['Years_of_Operations'] = 2020 - data['Outlet_Establishment_Year']

In [37]:
#Now we transform out y value/what we want to predict. In this case Sales data

le = LabelEncoder()
y = transformed_df.Item_Outlet_Sales
y = le.fit_transform(y)

In [38]:
y = pd.DataFrame(y, columns = ['Item_Outet_Sales'])
transformed_df['Item_Outlet_Sales'] = y

In [39]:
transformed_df.dtypes
#all predictors and target are in supported dtype for estimator

Low Fat                      float64
Regular                      float64
Foods                        float64
Drinks                       float64
Non Perishable               float64
Supermarket Type1            float64
Grocery Store                float64
Supermarket Type3            float64
Supermarket Type2            float64
Tier 1                       float64
Tier 2                       float64
Tier 3                       float64
OUT049                       float64
OUT017                       float64
OUT010                       float64
OUT027                       float64
OUT046                       float64
OUT018                       float64
OUT045                       float64
OUT019                       float64
OUT013                       float64
OUT035                       float64
Item_Identifier               object
Item_Weight                  float64
Item_Visibility              float64
Item_MRP                     float64
Outlet_Establishment_Year      int64
O

In [40]:
transformed_df.head()

,Low Fat,Regular,Foods,Drinks,Non Perishable,Supermarket Type1,Grocery Store,Supermarket Type3,Supermarket Type2,Tier 1,Tier 2,Tier 3,OUT049,OUT017,OUT010,OUT027,OUT046,OUT018,OUT045,OUT019,OUT013,OUT035,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,source,Item_Outlet_Sales,Years_of_Operations
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,FDW58,20.750000,0.007565,107.8622,1999,1.0,train,1692,21
1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FDW14,8.300000,0.038428,87.3198,2007,1.0,train,1692,13
2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NCN55,14.600000,0.099575,241.7538,1998,1.0,train,1692,22
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FDQ58,7.315000,0.015388,155.0340,2007,1.0,train,1692,13
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,FDY38,12.792854,0.118599,234.2300,1985,1.0,train,1692,35


In [41]:
corr_mat = transformed_df.corr()
corr_mat >= 0.5

,Low Fat,Regular,Foods,Drinks,Non Perishable,Supermarket Type1,Grocery Store,Supermarket Type3,Supermarket Type2,Tier 1,Tier 2,Tier 3,OUT049,OUT017,OUT010,OUT027,OUT046,OUT018,OUT045,OUT019,OUT013,OUT035,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Item_Outlet_Sales,Years_of_Operations
Low Fat,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Regular,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Foods,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Drinks,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Non Perishable,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Supermarket Type1,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
Grocery Store,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Supermarket Type3,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
Supermarket Type2,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True
Tier 1,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False


In [42]:
y = transformed_df[['Item_Outlet_Sales']].values
X = transformed_df.drop(labels = ['Item_Outlet_Sales', 'Item_Identifier', 'source', 'Outlet_Establishment_Year'], axis = 'columns')
X = X.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [43]:
print('', X_train.shape, '\n', X_test.shape, '\n', y_train.shape, '\n', y_test.shape)

 (9942, 27) 
 (4262, 27) 
 (9942, 1) 
 (4262, 1)


In [44]:
#fitting the model with training data
lr = LinearRegression(normalize = True)
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [45]:
pred = lr.predict(X_test)
print(pred)

[[1599.]
 [1559.]
 [1940.]
 ...
 [2007.]
 [1697.]
 [1259.]]


In [46]:
lr.score(X_test, y_test)

0.39707599036741015

In [47]:
# The coefficients
print('Coefficients: \n', lr.coef_)
# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, pred))


Coefficients: 
 [[-6.09593875e+15 -6.09593875e+15  3.13462134e+15  3.13462134e+15
   3.13462134e+15 -6.43513271e+15 -5.15882423e+15 -1.07650799e+13
  -3.81875022e+15 -1.45566651e+15  6.15856828e+15  7.38844496e+14
   3.75675652e+15  4.25357529e+15 -2.67192931e+14  5.98267891e+14
  -2.94741394e+15 -2.71930117e+15 -6.19689348e+15 -1.75168339e+15
  -6.60945325e+14  4.97185713e+15 -3.01267881e+00 -3.33315504e+01
   4.97544238e+00  5.03900627e+15  2.96898092e+14]]
Mean squared error: 310243.35
Coefficient of determination: 0.40


In [48]:
#Exporting predictions to csv file
pred_df = pd.DataFrame(pred, columns = ['Predictions'])
pred_df.to_csv('prdicitions.csv', index = False)